<a href="https://colab.research.google.com/github/vaishakmk/TedTalk-Analysis/blob/main/dheeraj_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import pandas as pd


import os
import io
import codecs
import joblib
import numpy as np
from numpy import percentile
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
import re # for regular expressions
import pandas as pd 
pd.set_option("display.max_colwidth", 200) 
import string
import tensorflow as tf

import warnings 
warnings.filterwarnings("ignore")

%matplotlib inline

#Load dataframe

In [ ]:
speaker=pd.read_csv('/content/drive/MyDrive/ted_talk_dataset/speaker_data.csv')
talk=pd.read_csv('/content/drive/MyDrive/ted_talk_dataset/talk_data.csv')
transcript=pd.read_csv('/content/drive/MyDrive/ted_talk_dataset/transcript_data.csv')


#Speaker

In [ ]:
speaker.head()

,talk,speaker,speaker_title,speaker_occ,speaker_bio
0,Can you outsmart the apples and oranges fallacy?,Elizabeth Cox,NaN,NaN,NaN
1,The exploitation of US college athletes,Tim Nevius,NaN,College sports lawyer,Tim Nevius is a leading sports lawyer and college athlete advocate.
2,How does ultrasound work?,Jacques Abramowicz,NaN,NaN,NaN
3,"An honest history of an ancient and ""nasty"" word",Kate Lister,NaN,Sex historian,Kate Lister is a sex historian and lecturers at Leeds Trinity University.
4,The electrical blueprints that orchestrate life,Michael Levin,NaN,Bioelectric explorer,"Michael Levin's research could give rise to adaptive, self-repairing robotics and novel platforms for artificial intelligence."


In [ ]:
speaker.isnull().sum()

talk              141
speaker           145
speaker_title    4433
speaker_occ       789
speaker_bio       764
dtype: int64

In [ ]:
#remove rows with no talk 
speaker=speaker.dropna(subset=['talk'])

In [ ]:
speaker.isnull().sum()

talk                0
speaker             4
speaker_title    4292
speaker_occ       648
speaker_bio       623
dtype: int64

In [ ]:
speaker=speaker.drop(columns=['speaker_title'])

In [ ]:
#fill the null values for speaker occ with "unknown"
speaker['speaker_occ'] = speaker['speaker_occ'].fillna('Unknown')

In [ ]:
#fill the null values for speaker bio with "unknown"
speaker['speaker_bio'] = speaker['speaker_bio'].fillna('Unknown')

In [ ]:
speaker.isnull().sum()

talk           0
speaker        4
speaker_occ    0
speaker_bio    0
dtype: int64

In [ ]:
#fill the null values for speaker with "unknown"
speaker['speaker'] = speaker['speaker'].fillna('Unknown')

In [ ]:
speaker.isnull().sum()

talk           0
speaker        0
speaker_occ    0
speaker_bio    0
dtype: int64

In [ ]:
speaker.head()

,talk,speaker,speaker_occ,speaker_bio
0,Can you outsmart the apples and oranges fallacy?,Elizabeth Cox,Unknown,Unknown
1,The exploitation of US college athletes,Tim Nevius,College sports lawyer,Tim Nevius is a leading sports lawyer and college athlete advocate.
2,How does ultrasound work?,Jacques Abramowicz,Unknown,Unknown
3,"An honest history of an ancient and ""nasty"" word",Kate Lister,Sex historian,Kate Lister is a sex historian and lecturers at Leeds Trinity University.
4,The electrical blueprints that orchestrate life,Michael Levin,Bioelectric explorer,"Michael Levin's research could give rise to adaptive, self-repairing robotics and novel platforms for artificial intelligence."


#Talk

In [ ]:
talk.head()

,talk_desc,event,talk_name,views,duration,tags,recorded_at,published on
0,It's 1997. The United States Senate has called a hearing about global warming. Some expert witnesses point out that past periods in Earth's history were warmer than the 20th century. Because such ...,TED-Ed,Can you outsmart the apples and oranges fallacy?,119359,327,"['education', 'psychology', 'animation', 'TED-Ed', 'brain', 'climate change']",2021-04-05T00:00:00.000+00:00,1617636753
1,"Colleges and universities in the US make billions of dollars each year from sports, compromising the health and education of athletes -- who are disproportionately Black -- in the name of money, p...",TEDxDayton,The exploitation of US college athletes,438573,611,"['sports', 'law', 'education', 'United States', 'race', 'policy', 'TEDx']",2020-11-10T00:00:00.000+00:00,1617634131
2,"In a dark cave, bats can't see much. But even with their eyes shut, they can navigate rocky topography at incredible speeds. This is because bats aren't just guided by their eyes, but rather, by t...",TED-Ed,How does ultrasound work?,140874,295,"['education', 'technology', 'animation', 'TED-Ed', 'invention', 'medicine', 'health care', 'medical imaging', 'science', 'pregnancy', 'innovation', 'health', 'biology', 'healthcare', 'sound']",2021-04-01T00:00:00.000+00:00,1617290223
3,"With candor and cunning, sex historian Kate Lister chronicles the curious journey of an ancient, honest word with innocent origins and a now-scandalous connotation in this uproarious love letter t...",TEDxUniversityofGlasgow,"An honest history of an ancient and ""nasty"" word",569477,1148,"['sex', 'language', 'ancient world', 'history', 'society', 'TEDx', 'humor']",2020-03-06T00:00:00.000+00:00,1617289565
4,"DNA isn't the only builder in the biological world -- there's also a mysterious bioelectric layer directing cells to work together to grow organs, systems and bodies, says biologist Michael Levin....",TED2020,The electrical blueprints that orchestrate life,122682,1176,"['biology', 'science', 'invention', 'robots', 'DNA', 'technology', 'nature', 'innovation', 'future', 'engineering']",2020-05-18T00:00:00.000+00:00,1617213773


In [ ]:
#no null values
talk.isnull().sum()

talk_desc       0
event           0
talk_name       0
views           0
duration        0
tags            0
recorded_at     0
published on    0
dtype: int64

#Transcript

In [ ]:
transcript.head()

,title,transcript
0,Can you outsmart the apples and oranges fallacy?,"Baking apple pie? Discount orange warehouse has you covered! A fruit’s a fruit, right?It’s 1988, and scientist James Hansen has just testified to the United States Congress that global warming t..."
1,The exploitation of US college athletes,"In college sports, American universities are exploiting, disproportionately, Black athletes for billions of dollars, while diminishing their education, health and safety. Let me start with a bit o..."
2,How does ultrasound work?,"In a pitch-black cave, bats can’t see much. But even with their eyes shut, they can navigate rocky topography at incredible speeds. This is because a bat’s flight isn’t just guided by its eyes, b..."
3,"An honest history of an ancient and ""nasty"" word","First, a warning. As far as offensive words go, you are now entering a hard-hat area. We're going to be unabashed in this, I am talking to you about a very particular word, a very powerful word, a..."
4,The electrical blueprints that orchestrate life,"Chris Anderson: Mike, welcome. It's good to see you. I'm excited for this conversation.Michael Levin: Thank you so much. I'm so happy to be here.CA: So, most of us have this mental model in biolog..."


In [ ]:
transcript.isnull().sum()

title           0
transcript    144
dtype: int64

In [ ]:
transcript = transcript.dropna(subset=['transcript'])

#Merging

In [ ]:
df = pd.merge(speaker, talk, how='inner', left_on = 'talk', right_on = 'talk_name')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4418 entries, 0 to 4417
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   talk          4418 non-null   object
 1   speaker       4418 non-null   object
 2   speaker_occ   4418 non-null   object
 3   speaker_bio   4418 non-null   object
 4   talk_desc     4418 non-null   object
 5   event         4418 non-null   object
 6   talk_name     4418 non-null   object
 7   views         4418 non-null   int64 
 8   duration      4418 non-null   int64 
 9   tags          4418 non-null   object
 10  recorded_at   4418 non-null   object
 11  published on  4418 non-null   int64 
dtypes: int64(3), object(9)
memory usage: 448.7+ KB


In [ ]:
df = df.drop(columns=['talk_name'])

In [ ]:
df = pd.merge(df, transcript, how='inner', left_on = 'talk', right_on = 'title')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7167 entries, 0 to 7166
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   talk          7167 non-null   object
 1   speaker       7167 non-null   object
 2   speaker_occ   7167 non-null   object
 3   speaker_bio   7167 non-null   object
 4   talk_desc     7167 non-null   object
 5   event         7167 non-null   object
 6   views         7167 non-null   int64 
 7   duration      7167 non-null   int64 
 8   tags          7167 non-null   object
 9   recorded_at   7167 non-null   object
 10  published on  7167 non-null   int64 
 11  title         7167 non-null   object
 12  transcript    7167 non-null   object
dtypes: int64(3), object(10)
memory usage: 783.9+ KB


In [ ]:
df = df.drop(columns=['title'])

In [ ]:
df = df.drop_duplicates(subset=['talk'])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4016 entries, 0 to 7166
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   talk          4016 non-null   object
 1   speaker       4016 non-null   object
 2   speaker_occ   4016 non-null   object
 3   speaker_bio   4016 non-null   object
 4   talk_desc     4016 non-null   object
 5   event         4016 non-null   object
 6   views         4016 non-null   int64 
 7   duration      4016 non-null   int64 
 8   tags          4016 non-null   object
 9   recorded_at   4016 non-null   object
 10  published on  4016 non-null   int64 
 11  transcript    4016 non-null   object
dtypes: int64(3), object(9)
memory usage: 407.9+ KB


In [ ]:
df.to_csv('/content/drive/MyDrive/ted_talk_dataset/dheeraj_data.csv',index=False)

#Analysis

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/ted_talk_dataset/dheeraj_data.csv')

In [ ]:
df1['transcript']

0       Baking apple pie? Discount orange warehouse has you covered! A fruit’s a fruit, right?It’s 1988, and scientist James Hansen  has just testified to the United States Congress  that global warming t...
1       In college sports, American universities are exploiting, disproportionately, Black athletes for billions of dollars, while diminishing their education, health and safety. Let me start with a bit o...
2       In a pitch-black cave, bats can’t see much. But even with their eyes shut, they can navigate rocky topography  at incredible speeds. This is because a bat’s flight isn’t just guided by its eyes, b...
3       First, a warning. As far as offensive words go, you are now entering a hard-hat area. We're going to be unabashed in this, I am talking to you about a very particular word, a very powerful word, a...
4       Chris Anderson: Mike, welcome. It's good to see you. I'm excited for this conversation.Michael Levin: Thank you so much. I'm so happy to be here.CA: So, most of

In [ ]:
#Number of characters 
def count_chars(text):
    return len(text)
    print(text)

df1['char_count'] = df1["transcript"].apply(lambda x:count_chars(x))

In [ ]:
df1['char_count']

0        4061
1        9091
2        4068
3       17155
4       19537
        ...  
4011    17403
4012    17454
4013    18249
4014    18317
4015    11814
Name: char_count, Length: 4016, dtype: int64

In [ ]:
#Number of words
def count_words(text):
    return len(text.split())

df1['word_count'] = df1["transcript"].apply(lambda x:count_words(x))
df1['word_count']

0        645
1       1444
2        654
3       3033
4       3451
        ... 
4011    3127
4012    3066
4013    3015
4014    3253
4015    2089
Name: word_count, Length: 4016, dtype: int64

In [ ]:
#Number of words in quotes
def count_words_in_quotes(text):
    x = re.findall("'.'|\".\"", text)
    count=0
    if x is None:
        return 0
    else:
        for i in x:
            t=i[1:-1]
            count+=count_words(t)
        return count

df1['quoted_word_count'] = df1["transcript"].apply(lambda x:count_words_in_quotes(x))

In [ ]:
df1['quoted_word_count']

0       0
1       0
2       0
3       0
4       0
       ..
4011    0
4012    0
4013    0
4014    1
4015    0
Name: quoted_word_count, Length: 4016, dtype: int64

In [ ]:
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
#Number of sentences
def count_sent(text):
    return len(nltk.sent_tokenize(text))

df1['sent_count'] = df1["transcript"].apply(lambda x:count_sent(x))

In [ ]:
df1['sent_count']

0        30
1        49
2        31
3       185
4       148
       ... 
4011    227
4012    255
4013    167
4014    273
4015    134
Name: sent_count, Length: 4016, dtype: int64

In [ ]:
#Count the number of unique words
def count_unique_words(text):
    return len(set(text.split()))

df1['unique_word_count'] = df1["transcript"].apply(lambda x:count_unique_words(x))
df1['unique_word_count']

0        374
1        697
2        399
3       1272
4       1034
        ... 
4011     989
4012    1153
4013    1330
4014    1291
4015     875
Name: unique_word_count, Length: 4016, dtype: int64

In [ ]:
#Count of stopwords
from nltk.corpus import stopwords
nltk.download('stopwords')

def count_stopwords(text):
    stop_words = set(stopwords.words('english'))  
    word_tokens = word_tokenize(text)
    stopwords_x = [w for w in word_tokens if w in stop_words]
    return len(stopwords_x)

df1['stopword_count'] = df1["transcript"].apply(lambda x:count_stopwords(x))
df1['stopword_count']

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0        296
1        591
2        231
3       1396
4       1731
        ... 
4011    1470
4012    1480
4013    1271
4014    1398
4015     986
Name: stopword_count, Length: 4016, dtype: int64

In [ ]:
#Calculating average word length
df1['avg_wordlength'] = df1['char_count']/df1['word_count']
df1['avg_wordlength']

0       6.296124
1       6.295706
2       6.220183
3       5.656116
4       5.661258
          ...   
4011    5.565398
4012    5.692759
4013    6.052736
4014    5.630802
4015    5.655337
Name: avg_wordlength, Length: 4016, dtype: float64

In [ ]:
#Calculating average sentence length
df1['avg_sentlength'] = df1['word_count']/df1['sent_count']
df1['avg_sentlength']

0       21.500000
1       29.469388
2       21.096774
3       16.394595
4       23.317568
          ...    
4011    13.775330
4012    12.023529
4013    18.053892
4014    11.915751
4015    15.589552
Name: avg_sentlength, Length: 4016, dtype: float64

In [ ]:
feature_set = df1

In [ ]:
feature_set.columns

Index(['talk', 'speaker', 'speaker_occ', 'speaker_bio', 'talk_desc', 'event',
       'views', 'duration', 'tags', 'recorded_at', 'published on',
       'transcript', 'char_count', 'word_count', 'quoted_word_count',
       'sent_count', 'unique_word_count', 'stopword_count', 'avg_wordlength',
       'avg_sentlength'],
      dtype='object')

In [ ]:
feature_set.drop(['talk', 'speaker', 'speaker_occ', 'speaker_bio', 'talk_desc', 'event',
       'views', 'duration', 'tags', 'recorded_at', 'published on',
       'transcript'],axis=1,inplace=True)


In [ ]:
df1.drop(['char_count', 'word_count', 'quoted_word_count',
       'sent_count', 'unique_word_count', 'stopword_count', 'avg_wordlength',
       'avg_sentlength'],axis=1,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4016 entries, 0 to 7166
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   talk          4016 non-null   object
 1   speaker       4016 non-null   object
 2   speaker_occ   4016 non-null   object
 3   speaker_bio   4016 non-null   object
 4   talk_desc     4016 non-null   object
 5   event         4016 non-null   object
 6   views         4016 non-null   int64 
 7   duration      4016 non-null   int64 
 8   tags          4016 non-null   object
 9   recorded_at   4016 non-null   object
 10  published on  4016 non-null   int64 
 11  transcript    4016 non-null   object
dtypes: int64(3), object(9)
memory usage: 407.9+ KB


In [ ]:
feature_set.head(5)

,char_count,word_count,quoted_word_count,sent_count,unique_word_count,stopword_count,avg_wordlength,avg_sentlength
0,4061,645,0,30,374,296,6.296124,21.500000
1,9091,1444,0,49,697,591,6.295706,29.469388
2,4068,654,0,31,399,231,6.220183,21.096774
3,17155,3033,0,185,1272,1396,5.656116,16.394595
4,19537,3451,0,148,1034,1731,5.661258,23.317568
